In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

import math
from typing import Any, Dict, List, Optional

import rasterio
import contextily
import geopandas
import haversine as hs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests
from tqdm.notebook import tqdm

from huhuha.settings import RAW_DATA_DIR, DATA_DIR
from huhuha.ops_tiles import get_otm_tile
from huhuha.utils import get_elevation, random_float

tqdm.pandas()
sns.set_theme(style='whitegrid')

In [3]:
df = pd.read_csv(DATA_DIR / 'weather' / '45_29__6_76__K.csv')

In [ ]:
temp,
wind_speed,
snow_1h

weather_main,
wind_deg,
rain_1h
clouds_all


(temp_min, temp_max)
pressure

In [10]:
df[['clouds_all', 'weather_main']].head(10)

,clouds_all,weather_main
0,0,Clear
1,0,Clear
2,0,Clear
3,7,Clear
4,27,Clouds
5,17,Clouds
6,23,Clouds
7,44,Clouds
8,76,Clouds
9,64,Clouds


In [9]:
df.columns

Index(['dt', 'dt_iso', 'timezone', 'city_name', 'lat', 'lon', 'temp',
       'visibility', 'dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure', 'sea_level', 'grnd_level', 'humidity', 'wind_speed',
       'wind_deg', 'wind_gust', 'rain_1h', 'rain_3h', 'snow_1h', 'snow_3h',
       'clouds_all', 'weather_id', 'weather_main', 'weather_description',
       'weather_icon'],
      dtype='object')

In [7]:
df.describe()

,dt,timezone,lat,lon,temp,visibility,dew_point,feels_like,temp_min,temp_max,...,humidity,wind_speed,wind_deg,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id
count,1.431420e+05,143142.000000,1.431420e+05,1.431420e+05,143142.000000,773.000000,143142.000000,143142.000000,143142.000000,143142.000000,...,143142.000000,143142.000000,143142.000000,216.000000,5778.000000,0.0,19968.000000,0.0,143142.000000,143142.000000
mean,1.385620e+09,5684.721465,4.529194e+01,6.756111e+00,269.048100,8998.188875,264.960196,267.553112,264.683226,272.869643,...,74.705817,1.330646,228.007992,2.967500,0.522785,NaN,0.593249,NaN,59.631918,761.773861
std,1.487108e+08,1777.345143,2.131636e-14,3.552726e-15,8.614693,2549.580242,7.516548,9.464122,8.641077,8.737424,...,19.007092,0.655289,91.226340,1.116957,0.602722,NaN,0.615259,NaN,38.188658,88.199578
min,1.128125e+09,3600.000000,4.529194e+01,6.756111e+00,238.880000,1000.000000,235.590000,234.210000,233.460000,242.380000,...,5.000000,0.000000,0.000000,0.400000,0.100000,NaN,0.100000,NaN,0.000000,500.000000
25%,1.256826e+09,3600.000000,4.529194e+01,6.756111e+00,262.500000,10000.000000,259.500000,260.400000,258.130000,266.330000,...,61.000000,0.860000,143.000000,2.007500,0.160000,NaN,0.190000,NaN,21.000000,800.000000
50%,1.385584e+09,7200.000000,4.529194e+01,6.756111e+00,268.780000,10000.000000,266.040000,267.600000,264.520000,272.630000,...,80.000000,1.260000,270.000000,3.135000,0.290000,NaN,0.380000,NaN,70.000000,802.000000
75%,1.514411e+09,7200.000000,4.529194e+01,6.756111e+00,275.560000,10000.000000,270.960000,275.000000,271.060000,279.620000,...,91.000000,1.730000,296.000000,3.810000,0.650000,NaN,0.760000,NaN,98.000000,804.000000
max,1.643238e+09,7200.000000,4.529194e+01,6.756111e+00,294.330000,10000.000000,284.560000,293.370000,294.330000,306.390000,...,100.000000,12.300000,360.000000,5.110000,7.050000,NaN,6.630000,NaN,100.000000,804.000000
